 I'd need a proper data set to do a fair comparison
 but i couldnt find a simple data set which was good for
 naive bayes within a few google searches..

In [1]:
from numpy.random import randn
from numpy import array, concatenate, zeros, ones
from sklearn.model_selection import train_test_split

data = []
labels = []
with open("wineX.txt") as data_file:
    for line in data_file.read().split("\n"):
        l = []
        for item in line.split(","):
            if item != "":
                l.append(float(item))
            
        if len(l) > 5:
            data.append(l)
        

with open("wineY.txt") as label_file:
    tmp = label_file.readlines()
    
    for label in tmp:
        labels.append(int(label[0]))
    
    
traind, testd, trainl, testl = train_test_split(data, labels, test_size=0.25)




In [2]:
from naive_bayes import classifier
from time import time
from numpy import arange, array

traind = array(traind)
trainl = array(trainl)

testd = array(testd)
testl = array(testl)

c = classifier()

timestamp = time()
c.train(traind, trainl)
print("Time to train: {}".format(time() - timestamp))

timestamp = time()
correct = 0
for index, t in enumerate(testd):
    correct += (c.predict(t) == testl[index])

print("Correct rate: {}".format(correct / len(testd)))
print("Avarage predict rate: {}".format((time() - timestamp) / len(testd)))

Time to train: 0.0007300376892089844
Correct rate: 0.9555555555555556
Avarage predict rate: 8.808771769205729e-05
